<div style="text-align: center">
<h2><font color="#004D7F" size=6>Modelado de datos</font></h2>
<h1></h1>

<div style="text-align: center">
<font color="#004D7F" size=5>Curso: Inteligencia Artificial</font><br>
<font color="#004D7F" size=5>Docente: Juan Villegas Cubas</font><br>
    <font color="#004D7F" size=5>Alumno: Gonzalo López Guerrero</font><br>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Noviembre 2020</font><br>

In [17]:
import pandas as pd
import numpy as np
%matplotlib inline
import plotly 
from wordcloud import WordCloud,STOPWORDS
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import math

#import twitter data
filename = '../Data/Tweets.csv'
dataset = pd.read_csv(filename)
print(dataset.shape)#rows x columns

(14640, 15)


In [18]:
# cantidad de tweet por clase
dataset['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [19]:
# convertir a entero los valores de las clases
dataset['airline_sentiment'] = dataset['airline_sentiment'].replace('neutral', 1)
dataset['airline_sentiment'] = dataset['airline_sentiment'].replace('negative', 0)
dataset['airline_sentiment'] = dataset['airline_sentiment'].replace('positive', 2)

In [20]:
# Dividar los datos entre text y clase
X = dataset['text'] # data
y = dataset['airline_sentiment'] # labels

<h3>Procesando Datos<7h3>

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer  #reemplazar los caracteres especiales
from tensorflow.keras.preprocessing.text import text_to_word_sequence #analiza la secuencia de los caracteres
# Convertisa los datos en token
# create tokenizer
t = Tokenizer()
t.fit_on_texts(X)
# Encuentra la cantidad de palabras únicas en los tweets
vocab_size = len(t.word_index) + 1
# indexar las palabras (relacionar con su par entero)
sequences = t.texts_to_sequences(X)

In [22]:
# Encuentra el tweet con mayor numero de caracteres para ser tomado como referencias 
#al momento de vectorizar a las palabras
def max_tweet():
    for i in range(1, len(sequences)):
        max_length = len(sequences[0])
        if len(sequences[i]) > max_length:
            max_length = len(sequences[i])
    return max_length
tweet_num = max_tweet()
print("Mayor numero de caracteres en un tweet " , tweet_num)

Mayor numero de caracteres en un tweet  30


In [23]:
# las palabras no encontradas en el tweet se llenan con ceros en el array
# https://realpython.com/python-keras-text-classification/
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = tweet_num
padded_X = pad_sequences(sequences, padding='post', maxlen=maxlen)
valor_maximo =np.amax(padded_X) 
padded_X

array([[  81,   62, 6686, ...,    0,    0,    0],
       [  81,  558,  590, ...,    0,    0,    0],
       [  81,    3,  207, ...,    0,    0,    0],
       ...,
       [  13,   75,  661, ...,    0,    0,    0],
       [  13,    6,   22, ...,    0,    0,    0],
       [  13,   41,   22, ...,    2,  179,    8]])

In [24]:
# convertir las etiquestas
labels = to_categorical(np.asarray(y))
labels
# 1 => representa la clase que representa negativo / neutral / positivo

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [25]:
array = dataset.values
padded_Y = array[:,1] #array de clases
padded_Y_test=padded_Y.astype('int')
#padded_Y
padded_X_test = (padded_X /valor_maximo) -1

In [26]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_X, labels, test_size = 0.2, random_state = 0)
# Size of train and test datasets
print('X_train size:', X_train.shape)
print('y_train size:', y_train.shape)
print('X_test size:', X_test.shape)
print('y_test size:', y_test.shape)

X_train size: (11712, 30)
y_train size: (11712, 3)
X_test size: (2928, 30)
y_test size: (2928, 3)


## Pre-trained word embedding

In [27]:
# GloVe se define como un “algoritmo de aprendizaje no supervisado 
#para obtener representaciones vectoriales de palabras”. 
#Descargamos datos del sitio web vinculado y usamos específicamente las incrustaciones
#100-dimensionales de 400k palabras de Wikipedia en inglés en 2014. 
#Esto se representa en un archivo txt que debemos analizar para crear un
#índice que mapee las palabras a su representación vectorial.

# 100 dimensional version (embedding dimension)

#ayuda a en la decodificacion
#https://stackoverflow.com/questions/9233027/unicodedecodeerror-charmap-codec-cant-decode-byte-x-in-position-y-character
embeddings_index = dict()
f = open('../Data/glove/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Vectores de palabras cargados: %s' % len(embeddings_index))


Vectores de palabras cargados: 400000


In [28]:
# Se Obtiene todas las palabras únicas en nuestro conjunto de entrenamiento: índice de tokenizadores
# Se encuentra el vector de peso correspondiente en la incrustación de GloVe

# Se define el tamaño de la matriz de incrustación: número de palabras únicas x incrustación dim (100)
embedding_matrix = np.zeros((vocab_size, 100))

# llenado de la matrix
for word, i in t.word_index.items():  # dictionary
    embedding_vector = embeddings_index.get(word) # obtiene un vector incrustado de palabra de GloVe
    if embedding_vector is not None:
        # se agrega a la matrix
        embedding_matrix[i] = embedding_vector # cada fila de la matrix

#print(embedding_matrix)
embedding_matrix.shape

(15769, 100)

In [29]:
# Crea una capa de incrustación usando una matriz de incrustación
from tensorflow.keras.layers import Embedding

# la entrada es vocab_size, la salida es 100
# pesos de la matriz de incrustación, establezca entrenable = Falso || para q no se modifiquen los valores de Glove
embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix],
                           input_length = tweet_num, trainable=False)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import BatchNormalization

### Model 1: Simple LSTM Model with regularization, increase dimensionality

In [31]:
lstm_mod1 = Sequential()
lstm_mod1.add(embedding_layer)
lstm_mod1.add(LSTM(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5))
lstm_mod1.add(Dense(3, activation='softmax'))
lstm_mod1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_mod1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 1,943,239
Trainable params: 366,339
Non-trainable params: 1,576,900
_________________________________________________________________


In [32]:
hist_1 = lstm_mod1.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 17s 468ms/step - loss: 0.8273 - acc: 0.6348 - val_loss: 0.7500 - val_acc: 0.6936
Epoch 2/100
37/37 [==============================] - 17s 463ms/step - loss: 0.6734 - acc: 0.7294 - val_loss: 0.6334 - val_acc: 0.7486
Epoch 3/100
37/37 [==============================] - 21s 571ms/step - loss: 0.6402 - acc: 0.7447 - val_loss: 0.6068 - val_acc: 0.7482
Epoch 4/100
37/37 [==============================] - 21s 577ms/step - loss: 0.6155 - acc: 0.7495 - val_loss: 0.6124 - val_acc: 0.7559
Epoch 5/100
31/37 [========================>.....] - ETA: 3s - loss: 0.5960 - acc: 0.7591

KeyboardInterrupt: 

In [ ]:
# Entrenamiento y test accuracy
loss, accuracy = lstm_mod1.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = lstm_mod1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

### Model 2: LSTM with regularization, reduce dimensionality

In [ ]:
lstm_mod2 = Sequential()
lstm_mod2.add(embedding_layer)
lstm_mod2.add(LSTM(64, 
               dropout = 0.2, 
               recurrent_dropout = 0.5))
lstm_mod2.add(Dense(3, activation='softmax'))
lstm_mod2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_mod2.summary()

In [ ]:
hist_2 = lstm_mod2.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

In [ ]:
# Find train and test accuracy
loss, accuracy = lstm_mod2.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = lstm_mod2.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

### Model 3: LSTM Layer Stacking

In [ ]:
# LSTM Model
model_3 = Sequential()
model_3.add(embedding_layer)
model_3.add(LSTM(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5,
                 return_sequences = True))
model_3.add(LSTM(128,
                dropout = 0.2,
                recurrent_dropout = 0.5))
model_3.add(Dense(3, activation='softmax'))
model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_3.summary()

In [ ]:
history_3 = model_3.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

In [ ]:
# Find train and test accuracy
loss, accuracy = model_3.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_3.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

### Model 4: GRU Layer Stacking

In [ ]:
# GRU Model 2:
model_4 = Sequential()
model_4.add(embedding_layer)
model_4.add(GRU(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5,
                 return_sequences = True))
model_4.add(GRU(128,
                dropout = 0.2,
                recurrent_dropout = 0.5))
model_4.add(Dense(3, activation='softmax'))
model_4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_4.summary()

In [ ]:
history_4 = model_4.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

In [ ]:
loss, accuracy = model_4.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_4.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

### Model 5: Reduced GRU with More Regularization

In [ ]:
model_5 = Sequential()
model_5.add(embedding_layer)
model_5.add(GRU(64, 
               dropout = 0.3, 
               recurrent_dropout = 0.5,
                 return_sequences = True))
model_5.add(GRU(32,
                dropout = 0.2,
                recurrent_dropout = 0.5))
model_5.add(Dense(3, activation='softmax'))
model_5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_5.summary()

In [ ]:
history_5 = model_5.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

In [ ]:
loss, accuracy = model_5.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_5.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

### Model 6: Bidirectional RNN

In [ ]:
from tensorflow.keras.layers import Bidirectional
# Bidirectional RNNs
model_6 = Sequential()
model_6.add(embedding_layer)
model_6.add(Bidirectional(LSTM(64,
                              dropout=0.2,
                              recurrent_dropout=0.5)))
model_6.add(Dense(3,activation='softmax'))
model_6.summary()


In [ ]:
model_6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history_6 = model_6.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

In [ ]:
loss, accuracy = model_6.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_6.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


# <font color="#004D7F">Comparación de Modelos</font>


La clave para una comparación equitativa de los algoritmos de ML es garantizar que cada algoritmo se evalúe de la misma manera en los mismos datos. Los algoritmos se comparan en un único conjunto de datos:
* Logistic Regression.
* Linear Discriminant Analysis.
* k-Nearest Neighbors.
* Classification and Regression Trees. 
* Naive Bayes.
* Support Vector Machines.

In [ ]:
models=[]
models.append(("Simple LSTM Model with regularization, increase dimensionality",  lstm_mod1.evaluate(X_train, y_train, verbose=False)))
models.append(("LSTM with regularization, reduce dimensionality",  lstm_mod2.evaluate(X_train, y_train, verbose=False)))
models.append(("LSTM Layer Stacking",  model_3.evaluate(X_train, y_train, verbose=False)))
models.append(("GRU Layer Stacking", model_4.evaluate(X_train, y_train, verbose=False)))
models.append(("Reduced GRU with More Regularization",  model_5.evaluate(X_train, y_train, verbose=False)))
models.append(("Bidirectional RNN",  model_6.evaluate(X_train, y_train, verbose=False)))

results=[]
names=[]


for name, model in models: 
    loss, accuracy = model
    results.append(accuracy)
    names.append(name)
    print(f"Accuracy: {name}: {cv_results.mean()*100.0:,.2f}%  ({cv_results.std()*100.0:,.2f}) %")